In [1]:
import pandas as pd

In [2]:
from urllib.parse import urlencode
import requests
# Чтобы загрузить данные в Jupyter Notebook, используем api Яндекс.Диска.
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

In [3]:
# в переменную public_key сохраним ссылку на Яндекс.Диска, где находятся данные для скачивания
public_key = 'https://disk.yandex.ru/d/Ta-weYfGM5CHjg' 

final_url = base_url + urlencode(dict(public_key=public_key)) 
data = pd.read_csv(requests.get(final_url).json()['href'])

In [4]:
data.head(2)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads


In [5]:
data.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

In [6]:
# перевод в дату
data.date = pd.to_datetime(data.date)

# MAU февраля:
Количество уникальных пользователей за месяц

In [9]:
# вытаскиваем из даты название месяца
data['month'] = data.date.dt.month_name()

In [10]:
# проверяем
data.head(2)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January


In [11]:
# посмотрим сколько всего месяцев есть в данных
data.month.unique()

array(['January', 'February', 'March'], dtype=object)

In [12]:
# найдем ответ, посчитаем кол-во уникальных
mau = data.query("month == 'February'").device_id.nunique()

In [13]:
print(f'MAU february: {mau}')

MAU february: 75032


# Количество установок в январе

In [14]:
january_install = data.query('month == "January" and event == "app_install"').shape[0]

In [15]:
print(f'January install: {january_install}')

January install: 80297


# Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей?

In [16]:
data.shape[0]

2747968

In [17]:
data.event.unique()

array(['app_start', 'choose_item', 'purchase', 'search', 'tap_basket',
       'app_install', 'register'], dtype=object)

In [18]:
# отдельно в таблицу запишем только покупки пользователей
purchases = data.query('event == "purchase"')

In [19]:
# так же, но только для установок
installs = data.query('event == "app_install"')

In [22]:
# в покупках оставим у пользователей дату только первой покупки
purchases = purchases.groupby('device_id', as_index=False).agg(date_purchase=('date', 'min'))

In [23]:
# переименуем колонку для удобства
installs = installs.rename(columns={'date': 'date_install'})

In [28]:
# объединим таблицы, так у пользователя будет дата установки и дата покупки
cohort_analysis = pd.merge(installs[['device_id', 'date_install']],
         purchases[['device_id', 'date_purchase']],
        on='device_id',
        how='left')

In [30]:
# посчитаем разницу по дням между установкой и первой покупкой
cohort_analysis['date_diff'] = (cohort_analysis.date_purchase - cohort_analysis.date_install).dt.days

In [35]:
cohort_analysis.head()

,device_id,date_install,date_purchase,date_diff
0,4921563,2020-01-01,2020-01-10,9.0
1,1311583,2020-01-01,NaT,NaN
2,23006325,2020-01-01,2020-01-01,0.0
3,5645652,2020-01-01,NaT,NaN
4,8798567,2020-01-01,NaT,NaN


In [32]:
# оставим только покупки, совершенные в первую неделю и запишем в новую таблицу
purchases_first_week = cohort_analysis.query('date_diff < 8')

In [36]:
# группируем пользователей по дате установки, это когорта (т.к. когорты по дате установки) и посчитаем всех в этой когорте
cohort_analysis = cohort_analysis.groupby('date_install', as_index=False) \
                .agg({'device_id': 'count'}) \
                .rename(columns={'device_id': 'installs'})

In [37]:
# так же сгруппируем всех по когорте, и таким образом посчитаем тех, кто совершил покупку в течении недели
purchases_first_week = purchases_first_week.groupby('date_install', as_index=False) \
                    .agg({'device_id': 'count'}) \
                    .rename(columns={'device_id': 'in_week'})

In [38]:
# соединим 2 таблицу по когортам
cohort_analysis = pd.merge(cohort_analysis, purchases_first_week)

In [39]:
# посчитаем конверсию
cohort_analysis['CR'] = cohort_analysis.in_week / cohort_analysis.installs

In [40]:
cohort_analysis

,date_install,installs,in_week,CR
0,2020-01-01,3579,1408,0.393406
1,2020-01-02,3144,1186,0.377226
2,2020-01-03,2402,834,0.347211
3,2020-01-04,1831,639,0.348990
4,2020-01-05,1671,587,0.351287
...,...,...,...,...
86,2020-03-27,1199,277,0.231026
87,2020-03-28,1091,209,0.191567
88,2020-03-29,1117,223,0.199642
89,2020-03-30,994,204,0.205231


In [41]:
# найдем когорту с самым высоким CR
cohort_analysis.iloc[cohort_analysis.CR.idxmax()]

date_install    2020-01-01 00:00:00
installs                       3579
in_week                        1408
CR                         0.393406
Name: 0, dtype: object

In [ ]:
round(cohort_analysis.CR.max() * 100, 1)

# С какого платного маркетингового канала пришло больше всего новых пользователей?

In [ ]:
best_add = data.drop_duplicates('device_id')

In [ ]:
best_add.device_id.nunique()

In [ ]:
best_add = best_add.query('utm_source != "-"')

In [ ]:
best_add.utm_source.value_counts().head(1).index

# Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

In [7]:
# это по всем, по уникальному device_id найдем распределение по событиям
# app_install не рассматриваем, так как на момент сбора данных пользователь мог скачать раньше и не попасть в таблицу
funnel_all = data.query('event != "app_install"') \
                .groupby('event', as_index=False) \
                .agg({'device_id': 'nunique'}) \
                .sort_values('device_id', ascending=False)

In [8]:
# в переменную сохраним максимальное число(это как раз и будет первый шаг пользователя), чтобы долю найти
first_step = funnel_all.device_id.max()

In [9]:
# находим долю
funnel_all['share'] = funnel_all.device_id.apply(lambda x: round(x / first_step * 100, 2))

In [10]:
# между переходом в корзину и регистрацией теряется больше всего пользователей
funnel_all

,event,device_id,share
0,app_start,190884,100.00
4,search,184488,96.65
1,choose_item,155691,81.56
5,tap_basket,125414,65.70
3,register,78310,41.02
2,purchase,70875,37.13


посмотрим теперь отдельно зарегестрированных и нет

In [17]:
# в отдельную переменную сохраняем дату регистрации каждого пользователя и device_id
registration = data.query('event == "register"').rename(columns={'date': 'reg_date'})[['device_id', 'reg_date']]

In [21]:
# объединим таблицы, чтобы создать колонку о дате регистрации для каждого пользователя
# если регистрации небыло, то будет пропущенное значение
funell_reg = pd.merge(data, registration, on='device_id', how='left')

In [29]:
# посмотрим на воронку не зарегестрированных, app_install не рассматриваем
funell_reg[funell_reg.reg_date.isna()].value_counts('event', ascending=False)

event
app_start      153983
search         143368
choose_item     98202
app_install     91071
tap_basket      55378
dtype: int64

In [34]:
# уберем незареганных
funell_reg = funell_reg.dropna(subset=['reg_date'])

In [46]:
# найдем разницу между событием и регистрацией, так как нам надо найти действия только зарегистрированных пользователелй
# если дата регистрации меньше даты события, то действие было после регистрации
# PS, можно просто знак меньше и не создавать колонку, проще будет

funell_reg['reg_diff'] = (funell_reg.reg_date - funell_reg.date).dt.days

In [52]:
# отфильтруем, и оставим только действия после регистрации
# не оставляем уникальных, тут почему-то так будет правильно, получается оставили все все действия
# если уников оставить, то будет число кол-ва зареганных почти во всех событиях
funell_reg.query('reg_diff < 1').event.value_counts()

app_start      519754
search         495411
choose_item    392783
tap_basket     295566
purchase       141383
register        78310
app_install     42448
Name: event, dtype: int64

# Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?
CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение

In [7]:
cr_analysis = data[['event', 'utm_source', 'device_id']].query('event == "app_start" or event == "purchase"')

In [12]:
cr_analysis.event.value_counts()

app_start    748705
purchase     141383
Name: event, dtype: int64

In [16]:
cr_table = cr_analysis.groupby(['event', 'utm_source'], as_index=False).agg({'device_id': 'nunique'})

In [20]:
cr_table = pd.pivot_table(cr_table, index='utm_source', columns='event', values='device_id')

In [25]:
cr_table['CR'] = round(cr_table.purchase / cr_table.app_start, 3)

In [26]:
cr_table

event,app_start,purchase,CR
utm_source,,,
-,52268,16598,0.318
facebook_ads,25957,9017,0.347
google_ads,38092,11339,0.298
instagram_ads,31045,10762,0.347
referal,15924,6362,0.400
vk_ads,34079,12364,0.363
yandex-direct,40707,12028,0.295


In [28]:
print(f'Пользователи, пришедшие с {cr_table.CR.idxmin()}, показали самую низкую конверсию в первую покупку')

Пользователи, пришедшие с yandex-direct, показали самую низкую конверсию в первую покупку


# Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [39]:
checks = data.query('event == "purchase" and utm_source != "-"')

In [40]:
checks = checks.drop_duplicates(subset=['device_id'])

In [42]:
checks = checks.groupby('utm_source').agg({'purchase_sum': 'median'})

In [44]:
print(f'Пользователи, пришедшие с {checks.purchase_sum.idxmax()}, имеют медианный первый чек выше')

Пользователи, пришедшие с referal, имеют медианный первый чек выше


# Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 
Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

In [53]:
# отфильтруем неизвестный канал и referal, сгруппируем по рекламному каналу, и посчитаем доход по каждой рекламе
romi = data.query('utm_source != "-" and utm_source != "referal"').groupby('utm_source').agg({'purchase_sum': 'sum'})

In [58]:
# добавим в таблицу данные о расходах по реклами, создадим новый столбец
romi['costs'] = pd.Series([8590498, 9553531, 10534878, 8561626, 10491707], 
          index=['facebook_ads', 'vk_ads', 'google_ads', 'instagram_ads', 'yandex-direct'])

In [62]:
romi

,purchase_sum,costs,ROMI
utm_source,,,
facebook_ads,12249901.0,8590498,0.426
google_ads,12868276.0,10534878,0.221
instagram_ads,14546969.0,8561626,0.699
vk_ads,16389652.5,9553531,0.716
yandex-direct,13915368.0,10491707,0.326


In [61]:
# найдем ROMI, (валовая прибыль(доход с продаж) минус расходы на маркетинг(рекламу)) делить на расходы на маркетинг(рекламу)
romi['ROMI'] = round((romi.purchase_sum - romi.costs) / romi.costs, 3)

In [63]:
print(f'{romi.ROMI.idxmax()} канал привлечения имеет самый высокий ROMI')

vk_ads канал привлечения имеет самый высокий ROMI
